## R_prime =/= R

In [1]:
# user-defined parameters
NR_SIDES = 12    # base polygon
NR_LAYERS = 4   # no. of steps until top
S = 10          # length of polygon sides
H = 30          # height of structure
R_prime = 50    # vertical radius of circumscribed circle

# derived quantities
ALPHA = 2*m.pi/NR_SIDES                 # base polygonal angle
R = S/(2*m.sin(m.pi/NR_SIDES))          # base radius of circumscribed circle
A = S/(2*m.tan(m.pi/NR_SIDES))          # base polygon apothem
GAMMA = np.asin(H/R_prime)              # vertical circumscribed circular arc
BETA = GAMMA/NR_LAYERS                  # vertical polygonal angle
S_prime = 2*R_prime*m.sin(0.5*BETA)     # vertical polygon side length
# A_prime = S_prime/(2*m.tan(m.pi/(NR_LAYERS))) # vertical polygon apothem (work in progress)

print(f"{NR_SIDES=}\n{NR_LAYERS=}\n{S=}\n{H=}\n{R_prime=}\n{ALPHA=}\n{R=}\n{A=}\n{GAMMA=}\n{BETA=}\n{S_prime=}")

NameError: name 'm' is not defined

In [ ]:
def idx(i, j):
    """Index mapping between angles and cartesian points"""
    return i%NR_SIDES + NR_SIDES*j%(NR_SIDES*(NR_LAYERS+1))

def create_polys(angle_offset):
    # get all angular coordinates in a suitably formatted input array
    alphas = np.linspace(angle_offset, 2*np.pi + angle_offset, NR_SIDES, endpoint=False)
    betas = np.linspace(0, GAMMA, (NR_LAYERS+1), endpoint=True)
    X, Y = np.meshgrid(alphas, betas)
    angles = np.array([X.flatten(), Y.flatten()]).T

    # calculate actual (cartesian) coordinates of vertices
    xyz = np.zeros((angles.shape[0], 3))
    xyz[:, 0] = (R - R_prime*(1 - np.cos(angles[:,1])))*np.cos(angles[:,0])
    xyz[:, 1] = (R - R_prime*(1 - np.cos(angles[:,1])))*np.sin(angles[:,0])
    xyz[:, 2] = R_prime*np.sin(angles[:,1])

    # define polygon faces for the 3D plotter to work with
    polys = np.zeros((NR_SIDES*NR_LAYERS, 5, 3))

    k = 0
    for j in range(betas.shape[0]-1): # we have (NR_LAYERS+1) beta angles, but only NR_LAYERS faces in that direction
        for i in range(alphas.shape[0]):
            polys[k, 0, :] = xyz[idx(i, j), :]
            polys[k, 1, :] = xyz[idx(i+1, j), :]
            polys[k, 2, :] = xyz[idx(i+1, j+1), :]
            polys[k, 3, :] = xyz[idx(i, j+1), :]
            polys[k, 4, :] = polys[k, 0, :] # add first vertex again to close the loop
            
            k += 1
    
    return polys

In [ ]:
# rotate object by angle
angle_offset = 0#.5*m.pi + 0.2*ALPHA

polys = create_polys(angle_offset)

In [ ]:
fig, axs = plt.subplots(1,2)

# TOP VIEW PROJECTION
poly = PolyCollection(polys[:,:,:2], alpha=0.7)
poly.set_facecolor('white')
poly.set_edgecolor('black')

axs[0].add_collection(poly)

lim = 1.1*R
axs[0].set_xlim(-lim, lim)
axs[0].set_ylim(-lim, lim)
axs[0].set_aspect('equal')

# SIDE VIEW PROJECTION
poly = PolyCollection(polys[:,:,0::2], alpha=0.7)
poly.set_facecolor('white')
poly.set_edgecolor('black')
axs[1].add_collection(poly)

lim = 1.1*R
axs[1].set_xlim(-lim, lim)
axs[1].set_ylim(0, 2*H)
axs[1].set_aspect('equal')

fig.show()